In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import seaborn as sns
import matplotlib as mpl
import os
from datetime import datetime
import missingno as msno

from sklearn import base
from sklearn.model_selection import KFold

from IPython.core.pylabtools import figsize
from scipy import stats

In [3]:
#Getting Working Directory
os.getcwd()

#Setting style and font size for graphs
sns.set_style("whitegrid")
mpl.rcParams['font.size'] = 13

In [4]:
%%time
#load original data into dataframe
train_file ='D:\\Abhishek\\Personal\\Springboard\\music_recom\\train.csv'
test_file ='D:\\Abhishek\\Personal\\Springboard\\music_recom\\test.csv'
songs_file ='D:\\Abhishek\\Personal\\Springboard\\music_recom\\songs.csv'
members_file ='D:\\Abhishek\\Personal\\Springboard\\music_recom\\members.csv'
songs_ex_file ='D:\\Abhishek\\Personal\\Springboard\\music_recom\\song_extra_info.csv'
train = pd.read_csv(train_file)#, na_values = '?')
test = pd.read_csv(test_file)
songs = pd.read_csv(songs_file)
members = pd.read_csv(members_file)
songs_ex = pd.read_csv(songs_ex_file)

Wall time: 24.2 s


In [60]:


matrix = train[0:20].drop(train.columns.difference(['msno','song_id','target']),axis = 1)

n_users = matrix.msno.unique().shape[0]
n_items = matrix.song_id.unique().shape[0]

users = pd.DataFrame(matrix.msno.unique(), columns = ['msno'])
users['index'] = users.index
users.set_index('msno', inplace = True)
users = users.to_dict()['index']
matrix['msno_index'] =  matrix.msno.map(users)

songs = pd.DataFrame(matrix.song_id.unique(), columns = ['song'])
songs['index'] = songs.index
songs.set_index('song', inplace = True)
songs = songs.to_dict()['index']
matrix['song_index'] =  matrix.song_id.map(songs)

#users = {users.msno: users.index}
matrix= matrix.drop(['msno', 'song_id'], axis = 1)
matrix

,target,msno_index,song_index
0,1,0,0
1,1,1,1
2,1,1,2
3,1,1,3
4,1,0,4
5,1,0,5
6,1,1,6
7,1,0,7
8,1,2,8
9,1,2,9


In [61]:
data_matrix = np.zeros((n_users, n_items))
for line in matrix.itertuples():
    print(line)
    data_matrix[line[2], line[3]] = line[1]

Pandas(Index=0, target=1, msno_index=0, song_index=0)
Pandas(Index=1, target=1, msno_index=1, song_index=1)
Pandas(Index=2, target=1, msno_index=1, song_index=2)
Pandas(Index=3, target=1, msno_index=1, song_index=3)
Pandas(Index=4, target=1, msno_index=0, song_index=4)
Pandas(Index=5, target=1, msno_index=0, song_index=5)
Pandas(Index=6, target=1, msno_index=1, song_index=6)
Pandas(Index=7, target=1, msno_index=0, song_index=7)
Pandas(Index=8, target=1, msno_index=2, song_index=8)
Pandas(Index=9, target=1, msno_index=2, song_index=9)
Pandas(Index=10, target=1, msno_index=2, song_index=10)
Pandas(Index=11, target=1, msno_index=2, song_index=11)
Pandas(Index=12, target=0, msno_index=3, song_index=12)
Pandas(Index=13, target=1, msno_index=4, song_index=13)
Pandas(Index=14, target=1, msno_index=3, song_index=14)
Pandas(Index=15, target=1, msno_index=4, song_index=15)
Pandas(Index=16, target=1, msno_index=3, song_index=16)
Pandas(Index=17, target=1, msno_index=3, song_index=17)
Pandas(Index

In [63]:
U, S, Vt = np.linalg.svd(data_matrix)

In [64]:
U

array([[0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])

In [65]:
S

array([2.        , 2.        , 2.        , 1.73205081, 1.41421356,
       1.        ])

In [67]:
Vt

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.5       ,  0.5       ,
         0.5       ,  0.5       ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.5       ,  0.5       ,  0.5       ,  0.        ,
         0.        ,  0.5       ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.5       ,  0.        ,  0.        ,  0.        ,  0.5       ,
         0.5       ,  0.        ,  0.5       ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  

In [57]:
from scipy.sparse import csc_matrix, vstack
x = csc_matrix((matrix.target, (matrix.msno_index, matrix.song_index)),(n_users,n_items))
x

<6x20 sparse matrix of type '<class 'numpy.int64'>'
	with 20 stored elements in Compressed Sparse Column format>

In [58]:
print(x)

  (0, 0)	1
  (1, 1)	1
  (1, 2)	1
  (1, 3)	1
  (0, 4)	1
  (0, 5)	1
  (1, 6)	1
  (0, 7)	1
  (2, 8)	1
  (2, 9)	1
  (2, 10)	1
  (2, 11)	1
  (3, 12)	0
  (4, 13)	1
  (3, 14)	1
  (4, 15)	1
  (3, 16)	1
  (3, 17)	1
  (3, 18)	0
  (5, 19)	1
